# LLM을 이용한 음식 카테고리 라벨링
갖고 있는 식약처 음식 데이터에는 종류가 나눠저 있지 않아서 LLM을 이용해 데이터를 추가해보자!

## 접근 방법
각 Agent들이 분업하여 카테고리의 정확도를 높혀보자.

    Categorizer: 
        input: 음식 이름, 프롬프트
        output: {"category_id": "카테고리 번호", "reason": "왜 그렇게 생각했는지 서술"}
        기능: 실제로 음식을 분류하는 역할
    Evaluator:
        input: 음식, 카테고리 번호
        output: {"is_approve": True or False, "reason": "어떤 부분이 틀렸는지 서술"}
        기능: 프롬프트를 개선하기위한 근거 도출
    Optimizer:
        input: 프롬프트, Categorizor.output, Evaluator.output
        output: {"new_prompt": "조금 더 최적화된 프롬프트"}
        기능: 근거를 토대로 프롬프트 최적화



In [9]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import json

In [90]:
llm = ChatOllama(
    model="gemma3",
    temperature=0.0,  # 창의성 조절 (0.0 ~ 1.0)
    top_k=40,         # 다음 토큰 선택 시 고려할 상위 k개 토큰
    top_p=0.9,        # 누적 확률이 p를 넘지 않는 가장 작은 토큰 집합
    num_ctx=2048,     # 컨텍스트 윈도우 크기
    repeat_penalty=1.1,  # 반복 패널티 (1.0 이상)
    num_predict=512,  # 생성할 최대 토큰 수
)

In [136]:
category_list = [
    "밥/죽/면류",
    "국/찌개/탕류",
    "찜/조림류",
    "구이/볶음류",
    "튀김/전류",
    "무침/숙채/샐러드류",
    "김치/절임류",
    "빵/샌드위치/버거/피자류",
    "분식류",
    "후식/간식류",
    "음료",
    "기타"
 ]

## Categorizer

In [97]:
init_categorizer_prompt = """
넌 음식 분류 모델이야. 음식을 입력 받으면 다음 같은 형식을 맞춰서 한번만 출력해.
1. 밥/죽/면류 (Rice/Porridge/Noodles): 밥, 죽, 국수, 파스타 등 곡물/면이 주를 이루는 형태
2. 국/찌개/탕류 (Soups/Stews/Casseroles): 국물이 많은 형태
3. 찜/조림류 (Steamed/Braised Dishes): 찌거나 국물이 적게 졸여진 형태
4. 구이/볶음류 (Grilled/Stir-fried Dishes): 굽거나 기름에 볶아진 형태
5. 튀김/전류 (Fried/Pancake Dishes): 기름에 튀기거나 부쳐진 형태
6. 무침/숙채/샐러드류 (Seasoned Salads - Cooked/Raw): 데치거나 익힌 채소 등을 양념에 버무리거나, 생채소에 드레싱을 곁들인 형태
7. 김치/절임류 (Kimchi/Pickles): 발효되거나 절여진 형태의 반찬
8. 빵/샌드위치/버거/피자류 (Bread/Sandwiches/Burgers/Pizzas): 빵이나 도우가 주를 이루는 형태 (인기 품목은 별도 L1으로 분리)
9. 분식류 (Snack Foods - Korean Style): 한국 길거리/분식집에서 인기 있는 특정 품목 모음
10. 후식/간식류 (Desserts/Snacks): 달콤하거나 식사 외에 가볍게 먹는 형태
11. 음료 (Beverages): 마시는 것
12. 기타 (Others): 위 카테고리에 속하기 어려운 품목 (예: 과일 단품, 마른 안주 등)
입력: {food_name}
출력: \'{{category_id:1, reason:카테고리를 선택한 이유 3줄 정도}}\'
"""
# 출력: \'{{'category_id': '1', 'reason': '카테고리를 선택한 이유 3줄 정도'}}\'
categorizer_prompt = PromptTemplate.from_template(init_categorizer_prompt)

In [98]:
categorizer = (categorizer_prompt | llm)

In [177]:
food_name = "덮밥_낙지"
cls_response = categorizer.invoke({"food_name": food_name})

### 출력 통제 과정

### 최종 출력

In [179]:
cls_result = json.loads(cls_response.content.replace('`', '').replace('json', ''))
cls_result["category_id"] = category_list[cls_result["category_id"]-1]
cls_result

{'category_id': '밥/죽/면류',
 'reason': '낙지는 밥과 함께 제공되는 경우가 많고, 덮밥은 밥을 주재료로 하는 요리이므로 밥/죽/면류(Rice/Porridge/Noodles) 카테고리에 속합니다. 또한, 덮밥은 밥 위에 다양한 재료를 올려 먹는 형태로, 밥을 주재료로 하는 요리라는 점에서 해당 카테고리에 적합합니다. 덮밥은 밥을 주재료로 하는 요리라는 점에서 해당 카테고리에 적합합니다.'}

## Evaluator

In [194]:
init_evaluator_prompt = """
넌 객관적인 모델 평가자야. 분류해야할 음식 이름과 분류모델의 출력을 입력 받아서 형식에 맞게 출력해.
ex)
입력: 돈까스->덮밥_낙지->{{'category_id': 밥/죽/면류, 'reason': '낙지는 밥과 함께 제공되는 경우가 많고, 덮밥은 밥을 주재료로 하는 요리이므로 밥/죽/면류(Rice/Porridge/Noodles) 카테고리에 속합니다. 또한, 덮밥은 밥 위에 다양한 재료를 올려 조리하는 형태이므로 이 카테고리 분류에 적합합니다. 덮밥은 밥을 주재료로 하는 요리이므로 이 카테고리 분류에 적합합니다.'}}
출력: \'{{is_approve:true, reason: '덮밥은 밥을 주재료로 하는 요리라는 표현이 정확하고 최종적으로 분류가 틀리지 않았습다.'}}\'

입력: {food_name}->{cls_result}
출력: 
"""
evaluator_prompt = PromptTemplate.from_template(init_evaluator_prompt)

In [195]:
evaluator = (evaluator_prompt | llm)

In [196]:
eval_response = evaluator.invoke({"food_name": food_name, "cls_result": cls_result})

In [197]:
print(eval_response.content)

```json
{
  "is_approve": true,
  "reason": "덮밥은 밥을 주재료로 하는 요리라는 표현이 정확하고 최종적으로 분류가 틀리지 않았습니다."
}
```



In [199]:
eval_result = json.loads(eval_response.content.replace('`', '').replace('json', ''))
eval_result

{'is_approve': True,
 'reason': '덮밥은 밥을 주재료로 하는 요리라는 표현이 정확하고 최종적으로 분류가 틀리지 않았습니다.'}

## Optimizer

In [200]:
init_optimizer_prompt = """
넌 프롬프트 최적화 모델이야. 아래 정보를 바탕으로 더 나은 음식 분류 프롬프트를 제안해.
- 기존 프롬프트: {old_prompt}
- Categorizer 결과: {categorizer_output}
- Evaluator 결과: {evaluator_output}

이 근거를 바탕으로, 더 정확한 분류가 가능하도록 프롬프트를 수정해서 new_prompt만 JSON으로 반환해.
예시: {{"new_prompt": "수정된 프롬프트 내용"}}
"""

optimizer_prompt = PromptTemplate.from_template(init_optimizer_prompt)

In [201]:
optimizer = (optimizer_prompt | llm)

In [202]:
opt_response = optimizer.invoke({
    "old_prompt": init_categorizer_prompt,
    "categorizer_output": cls_result,
    "evaluator_output": eval_result["reason"]
})

In [204]:
print(opt_response.content)

```json
{
  "new_prompt": "넌 음식 분류 전문가 모델이야. 음식 이름을 입력받아 다음 형식에 맞춰서 한 번만 출력해. 각 카테고리 선택 이유를 3줄 이내로 설명해야 해.\n\n1. 밥/죽/면류 (Rice/Porridge/Noodles): 밥, 죽, 국수, 파스타 등 곡물/면이 주를 이루는 형태\n2. 국/찌개/탕류 (Soups/Stews/Casseroles): 국물이 많은 형태\n3. 찜/조림류 (Steamed/Braised Dishes): 찌거나 국물이 적게 졸여진 형태\n4. 구이/볶음류 (Grilled/Stir-fried Dishes): 굽거나 기름에 볶아진 형태\n5. 튀김/전류 (Fried/Pancake Dishes): 기름에 튀기거나 부쳐진 형태\n6. 무침/숙채/샐러드류 (Seasoned Salads - Cooked/Raw): 데치거나 익힌 채소 등을 양념에 버무리거나, 생채소에 드레싱을 곁들인 형태\n7. 김치/절임류 (Kimchi/Pickles): 발효되거나 절여진 형태의 반찬\n8. 빵/샌드위치/버거/피자류 (Bread/Sandwiches/Burgers/Pizzas): 빵이나 도우가 주를 이루는 형태 (인기 품목은 별도 L1으로 분리)\n9. 분식류 (Snack Foods - Korean Style): 한국 길거리/분식집에서 인기 있는 특정 품목 모음\n10. 후식/간식류 (Desserts/Snacks): 달콤하거나 식사 외에 가볍게 먹는 형태\n11. 음료 (Beverages): 마시는 것\n12. 기타 (Others): 위 카테고리에 속하기 어려운 품목 (예: 과일 단품, 마른 안주 등)\n\n입력: {food_name}\n출력: '{{category_id:1, reason:카테고리를 선택한 이유 3줄 이내로 설명}'"
}
```


In [205]:
opt_result = json.loads(opt_response.content.replace('`', '').replace('json', ''))
opt_result

{'new_prompt': "넌 음식 분류 전문가 모델이야. 음식 이름을 입력받아 다음 형식에 맞춰서 한 번만 출력해. 각 카테고리 선택 이유를 3줄 이내로 설명해야 해.\n\n1. 밥/죽/면류 (Rice/Porridge/Noodles): 밥, 죽, 국수, 파스타 등 곡물/면이 주를 이루는 형태\n2. 국/찌개/탕류 (Soups/Stews/Casseroles): 국물이 많은 형태\n3. 찜/조림류 (Steamed/Braised Dishes): 찌거나 국물이 적게 졸여진 형태\n4. 구이/볶음류 (Grilled/Stir-fried Dishes): 굽거나 기름에 볶아진 형태\n5. 튀김/전류 (Fried/Pancake Dishes): 기름에 튀기거나 부쳐진 형태\n6. 무침/숙채/샐러드류 (Seasoned Salads - Cooked/Raw): 데치거나 익힌 채소 등을 양념에 버무리거나, 생채소에 드레싱을 곁들인 형태\n7. 김치/절임류 (Kimchi/Pickles): 발효되거나 절여진 형태의 반찬\n8. 빵/샌드위치/버거/피자류 (Bread/Sandwiches/Burgers/Pizzas): 빵이나 도우가 주를 이루는 형태 (인기 품목은 별도 L1으로 분리)\n9. 분식류 (Snack Foods - Korean Style): 한국 길거리/분식집에서 인기 있는 특정 품목 모음\n10. 후식/간식류 (Desserts/Snacks): 달콤하거나 식사 외에 가볍게 먹는 형태\n11. 음료 (Beverages): 마시는 것\n12. 기타 (Others): 위 카테고리에 속하기 어려운 품목 (예: 과일 단품, 마른 안주 등)\n\n입력: {food_name}\n출력: '{{category_id:1, reason:카테고리를 선택한 이유 3줄 이내로 설명}'"}

In [206]:
print(opt_result["new_prompt"])

넌 음식 분류 전문가 모델이야. 음식 이름을 입력받아 다음 형식에 맞춰서 한 번만 출력해. 각 카테고리 선택 이유를 3줄 이내로 설명해야 해.

1. 밥/죽/면류 (Rice/Porridge/Noodles): 밥, 죽, 국수, 파스타 등 곡물/면이 주를 이루는 형태
2. 국/찌개/탕류 (Soups/Stews/Casseroles): 국물이 많은 형태
3. 찜/조림류 (Steamed/Braised Dishes): 찌거나 국물이 적게 졸여진 형태
4. 구이/볶음류 (Grilled/Stir-fried Dishes): 굽거나 기름에 볶아진 형태
5. 튀김/전류 (Fried/Pancake Dishes): 기름에 튀기거나 부쳐진 형태
6. 무침/숙채/샐러드류 (Seasoned Salads - Cooked/Raw): 데치거나 익힌 채소 등을 양념에 버무리거나, 생채소에 드레싱을 곁들인 형태
7. 김치/절임류 (Kimchi/Pickles): 발효되거나 절여진 형태의 반찬
8. 빵/샌드위치/버거/피자류 (Bread/Sandwiches/Burgers/Pizzas): 빵이나 도우가 주를 이루는 형태 (인기 품목은 별도 L1으로 분리)
9. 분식류 (Snack Foods - Korean Style): 한국 길거리/분식집에서 인기 있는 특정 품목 모음
10. 후식/간식류 (Desserts/Snacks): 달콤하거나 식사 외에 가볍게 먹는 형태
11. 음료 (Beverages): 마시는 것
12. 기타 (Others): 위 카테고리에 속하기 어려운 품목 (예: 과일 단품, 마른 안주 등)

입력: {food_name}
출력: '{{category_id:1, reason:카테고리를 선택한 이유 3줄 이내로 설명}'


In [208]:
print(init_categorizer_prompt)


넌 음식 분류 모델이야. 음식을 입력 받으면 다음 같은 형식을 맞춰서 한번만 출력해.
1. 밥/죽/면류 (Rice/Porridge/Noodles): 밥, 죽, 국수, 파스타 등 곡물/면이 주를 이루는 형태
2. 국/찌개/탕류 (Soups/Stews/Casseroles): 국물이 많은 형태
3. 찜/조림류 (Steamed/Braised Dishes): 찌거나 국물이 적게 졸여진 형태
4. 구이/볶음류 (Grilled/Stir-fried Dishes): 굽거나 기름에 볶아진 형태
5. 튀김/전류 (Fried/Pancake Dishes): 기름에 튀기거나 부쳐진 형태
6. 무침/숙채/샐러드류 (Seasoned Salads - Cooked/Raw): 데치거나 익힌 채소 등을 양념에 버무리거나, 생채소에 드레싱을 곁들인 형태
7. 김치/절임류 (Kimchi/Pickles): 발효되거나 절여진 형태의 반찬
8. 빵/샌드위치/버거/피자류 (Bread/Sandwiches/Burgers/Pizzas): 빵이나 도우가 주를 이루는 형태 (인기 품목은 별도 L1으로 분리)
9. 분식류 (Snack Foods - Korean Style): 한국 길거리/분식집에서 인기 있는 특정 품목 모음
10. 후식/간식류 (Desserts/Snacks): 달콤하거나 식사 외에 가볍게 먹는 형태
11. 음료 (Beverages): 마시는 것
12. 기타 (Others): 위 카테고리에 속하기 어려운 품목 (예: 과일 단품, 마른 안주 등)
입력: {food_name}
출력: '{{category_id:1, reason:카테고리를 선택한 이유 3줄 정도}}'



## 조합

In [218]:
food_list = ["국밥", "덮밥", "토마토_스파게티", "신발"]

In [219]:
max_try = 3
fi = 0

In [220]:
opt_categorizer_prompt = init_categorizer_prompt

In [221]:
for food_name in food_list:
    for i in range(max_try):
        cls_response = (PromptTemplate.from_template(opt_categorizer_prompt) | llm).invoke({"food_name": food_name})
        cls_result = json.loads(cls_response.content.replace('`', '').replace('json', ''))
        cls_result["category_id"] = category_list[cls_result["category_id"]-1]
        
        eval_response = evaluator.invoke({"food_name": food_name, "cls_result": cls_result})
        eval_result = json.loads(eval_response.content.replace('`', '').replace('json', ''))
        
        if eval_result["is_approve"]:
            break
        
        opt_response = optimizer.invoke({
            "old_prompt": init_categorizer_prompt,
            "categorizer_output": cls_result,
            "evaluator_output": eval_result["reason"]
        })
        opt_result = json.loads(opt_response.content.replace('`', '').replace('json', ''))

        print(food_name, i, "번째")
        print("before prompt:", opt_categorizer_prompt)
        print("eval_reason:", eval_result["reason"])
        print("after prompt:", opt_result["new_prompt"])

        opt_categorizer_prompt = opt_result["new_prompt"]


신발 0 번째
before prompt: 
넌 음식 분류 모델이야. 음식을 입력 받으면 다음 같은 형식을 맞춰서 한번만 출력해.
1. 밥/죽/면류 (Rice/Porridge/Noodles): 밥, 죽, 국수, 파스타 등 곡물/면이 주를 이루는 형태
2. 국/찌개/탕류 (Soups/Stews/Casseroles): 국물이 많은 형태
3. 찜/조림류 (Steamed/Braised Dishes): 찌거나 국물이 적게 졸여진 형태
4. 구이/볶음류 (Grilled/Stir-fried Dishes): 굽거나 기름에 볶아진 형태
5. 튀김/전류 (Fried/Pancake Dishes): 기름에 튀기거나 부쳐진 형태
6. 무침/숙채/샐러드류 (Seasoned Salads - Cooked/Raw): 데치거나 익힌 채소 등을 양념에 버무리거나, 생채소에 드레싱을 곁들인 형태
7. 김치/절임류 (Kimchi/Pickles): 발효되거나 절여진 형태의 반찬
8. 빵/샌드위치/버거/피자류 (Bread/Sandwiches/Burgers/Pizzas): 빵이나 도우가 주를 이루는 형태 (인기 품목은 별도 L1으로 분리)
9. 분식류 (Snack Foods - Korean Style): 한국 길거리/분식집에서 인기 있는 특정 품목 모음
10. 후식/간식류 (Desserts/Snacks): 달콤하거나 식사 외에 가볍게 먹는 형태
11. 음료 (Beverages): 마시는 것
12. 기타 (Others): 위 카테고리에 속하기 어려운 품목 (예: 과일 단품, 마른 안주 등)
입력: {food_name}
출력: '{{category_id:1, reason:카테고리를 선택한 이유 3줄 정도}}'

eval_reason: 신발은 음식과 관련이 없는 물품이므로 음식 분류 모델의 범위를 벗어납니다. 음식과 관련된 품목을 분류하는 모델이 아니기 때문입니다. 또한, 신발은 식재료나 조리 과정에 사용되지 않으므로 음식 분류의 대상이 아닙니다.
after prompt: 넌 음식 분류 모델이야. 음식 이름을

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)